In [1]:
%run 基本定义.ipynb

# 0. 训练受污染的样本

In [2]:
## 使用transE模型，wn18rr数据集

## 可修改model参数，训练rotate模型
## 可修改data_path参数，指定训练的数据集
## 可修改fake参数，添加何种受污染的数据

## 在原有训练参数下，增加fake参数。该参数指定噪声样本的名字。

base_args={
    'cuda': True,
    'evaluate_train': False,
    'double_entity_embedding': False, 
    'double_relation_embedding': False, 
    'negative_sample_size': 1024, 
    'hidden_dim': 200, 
    'gamma': 6.0, 
    'negative_adversarial_sampling': True, 
    'adversarial_temperature': 0.5, 
    'regularization': 0.0, 
    'test_batch_size': 8, 
    'uni_weight': False, 
    'learning_rate': 0.0005,
    'cpu_num': 10, 
    'comments': '\n', 
}

args={
    'do_train': True, 
    'do_test': True, 
    'data_path': 'data/wn18rr', 
    'model': 'TransE', 
    'batch_size': 512, 
    'max_steps': 40000, ## 经验数据 RotatE FB15k-237 30000
                        ## RotatE wn18rr 30000
                        ## TransE FB15k-237 20000
                        ## TransE wn18rr 40000
    'warm_up_steps': 20000, 
    'no_save': True, 
    'save_path': './models/TransE_wn18rr_baseline', 
    'save_checkpoint_steps': 10000, 
    'valid_steps': 10000, 
    'log_steps': 2000, 
    'classify_steps': 5000, 
    'test_log_steps': 1000, 
    'nentity': 40943, 
    'nrelation': 11,
    'fake':'gs_l2',
    'do_valid':False
}
args = {**base_args,**args}
log_filename = "train"
set_logger(args, filename=log_filename)
input_data = get_input_data(args)
trainer = BaseTrainer.get_trainer(input_data, args)
kge_model = trainer.kge_model
logging.info('Start Training...')
logging.info(f"args is {args}")
logging.info('learning_rate = %f' % trainer.lr)
for step in range(args['max_steps']):
    trainer.basicTrainStep(step)
trainer.save_model()

2023-12-19 18:05:01,639 INFO     

2023-12-19 18:05:01,640 INFO     Model: TransE
2023-12-19 18:05:01,640 INFO     Data Path: data/wn18rr
2023-12-19 18:05:01,641 INFO     #entity: 40943
2023-12-19 18:05:01,642 INFO     #relation: 11
2023-12-19 18:05:01,642 INFO     #train: 86935	#valid: 3034	#test: 100
2023-12-19 18:05:08,453 INFO     Model Parameter Configuration:
2023-12-19 18:05:08,455 INFO     Parameter gamma: torch.Size([1]), require_grad = False
2023-12-19 18:05:08,455 INFO     Parameter embedding_range: torch.Size([1]), require_grad = False
2023-12-19 18:05:08,456 INFO     Parameter entity_embedding: torch.Size([40943, 200]), require_grad = True
2023-12-19 18:05:08,457 INFO     Parameter relation_embedding: torch.Size([11, 200]), require_grad = True
2023-12-19 18:05:08,458 INFO     Start Training...
2023-12-19 18:05:08,459 INFO     args is {'cuda': True, 'evaluate_train': False, 'double_entity_embedding': False, 'double_relation_embedding': False, 'negative_sample_size': 1024, '

# 1. 测试结果

In [3]:
logging.info('Evaluating on Test Dataset...')
metrics = kge_model.test_step(kge_model, input_data.test_triples, input_data.all_true_triples, args)
log_metrics('Test', args['max_steps'], metrics)

2023-12-19 18:16:41,625 INFO     Evaluating on Test Dataset...
2023-12-19 18:16:42,343 INFO     Evaluating the model... (0/26)
2023-12-19 18:16:43,590 INFO     len of triple2mode2ranking: 100
2023-12-19 18:16:43,600 INFO     Test MRR at step 40000: 0.420492
2023-12-19 18:16:43,601 INFO     Test MR at step 40000: 3.205000
2023-12-19 18:16:43,602 INFO     Test HITS@1 at step 40000: 0.095000
2023-12-19 18:16:43,602 INFO     Test HITS@3 at step 40000: 0.705000
2023-12-19 18:16:43,603 INFO     Test HITS@10 at step 40000: 0.990000
2023-12-19 18:16:43,603 INFO     Test 1_MR at step 40000: 1.000000
2023-12-19 18:16:43,604 INFO     Test 1_MRR at step 40000: 1.000000
2023-12-19 18:16:43,604 INFO     Test 1_NUM at step 40000: 12.000000
2023-12-19 18:16:43,605 INFO     Test 10_MR at step 40000: 3.266304
2023-12-19 18:16:43,607 INFO     Test 10_MRR at step 40000: 0.386775
2023-12-19 18:16:43,607 INFO     Test 10_NUM at step 40000: 184.000000
2023-12-19 18:16:43,608 INFO     Test 100_MR at step 4000

# 2. 对比结果

In [4]:
args['fake']='empty'
args['do_train'] = False
args['do_valid'] = False
args['init_checkpoint'] = args['save_path']
log_filename = 'empty'
dataset = args['data_path'].split("/")[-1]
args['save_path'] = "./models/%s_%s_baseline" % (args['model'], dataset)
set_logger(args, filename=log_filename)
input_data = get_input_data(args)
trainer = BaseTrainer.get_trainer(input_data, args)
trainer.load_model()
kge_model = trainer.kge_model
logging.info('Evaluating on Test Dataset...')
metrics = kge_model.test_step(kge_model, input_data.test_triples, input_data.all_true_triples, args)
log_metrics('Test', args['max_steps'], metrics)

2023-12-19 18:16:53,148 INFO     

2023-12-19 18:16:53,148 INFO     

2023-12-19 18:16:53,150 INFO     Model: TransE
2023-12-19 18:16:53,150 INFO     Model: TransE
2023-12-19 18:16:53,152 INFO     Data Path: data/wn18rr
2023-12-19 18:16:53,152 INFO     Data Path: data/wn18rr
2023-12-19 18:16:53,154 INFO     #entity: 40943
2023-12-19 18:16:53,154 INFO     #entity: 40943
2023-12-19 18:16:53,155 INFO     #relation: 11
2023-12-19 18:16:53,155 INFO     #relation: 11
2023-12-19 18:16:53,156 INFO     #train: 86835	#valid: 3034	#test: 100
2023-12-19 18:16:53,156 INFO     #train: 86835	#valid: 3034	#test: 100
2023-12-19 18:16:54,246 INFO     Model Parameter Configuration:
2023-12-19 18:16:54,246 INFO     Model Parameter Configuration:
2023-12-19 18:16:54,248 INFO     Parameter gamma: torch.Size([1]), require_grad = False
2023-12-19 18:16:54,248 INFO     Parameter gamma: torch.Size([1]), require_grad = False
2023-12-19 18:16:54,250 INFO     Parameter embedding_range: torch.Size([1]), require_gra

load model from ./models/TransE_wn18rr_baseline/checkpoint


2023-12-19 18:16:55,281 INFO     Evaluating the model... (0/26)
2023-12-19 18:16:55,281 INFO     Evaluating the model... (0/26)
2023-12-19 18:16:56,417 INFO     len of triple2mode2ranking: 100
2023-12-19 18:16:56,417 INFO     len of triple2mode2ranking: 100
2023-12-19 18:16:56,427 INFO     Test MRR at step 40000: 0.464333
2023-12-19 18:16:56,427 INFO     Test MRR at step 40000: 0.464333
2023-12-19 18:16:56,429 INFO     Test MR at step 40000: 2.595000
2023-12-19 18:16:56,429 INFO     Test MR at step 40000: 2.595000
2023-12-19 18:16:56,430 INFO     Test HITS@1 at step 40000: 0.120000
2023-12-19 18:16:56,430 INFO     Test HITS@1 at step 40000: 0.120000
2023-12-19 18:16:56,431 INFO     Test HITS@3 at step 40000: 0.795000
2023-12-19 18:16:56,431 INFO     Test HITS@3 at step 40000: 0.795000
2023-12-19 18:16:56,432 INFO     Test HITS@10 at step 40000: 1.000000
2023-12-19 18:16:56,432 INFO     Test HITS@10 at step 40000: 1.000000
2023-12-19 18:16:56,433 INFO     Test 1_MR at step 40000: 1.0000